In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [2]:
hist = utils.read_df_pkl('../input/historical_transactions0*')
new = utils.read_df_pkl('../input/new_mer*0*')
df = pd.concat([hist, new], axis=0)

100%|██████████| 3/3 [00:00<00:00,  3.56it/s]


In [5]:
# dm = pd.read_csv('../input/merchant_id_outlier_purchase_amount.csv')
# ### cnt_idが3000以上のmechant_id
# dm = dm[dm.cnt_id>=3000]
dm = df[[key, 'merchant_id']]
dm.head()

,card_id,merchant_id
0,C_ID_4e6213e9bc,M_ID_e020e9b302
1,C_ID_4e6213e9bc,M_ID_86ec983688
2,C_ID_4e6213e9bc,M_ID_979ed661fc
3,C_ID_4e6213e9bc,M_ID_e6d5ae8ea6
4,C_ID_4e6213e9bc,M_ID_e020e9b302


In [8]:
m_id_list = [ 'M_ID_00a6ca8a8a' ,'M_ID_076cd8a829' ,'M_ID_09d7a200d1' ,'M_ID_1ac6bbc867' ,'M_ID_1f4773aa76' ,'M_ID_2637773dd2' ,'M_ID_26d4fadb60' ,'M_ID_29ec8d2ab8' ,'M_ID_2c4b8c61f6' ,'M_ID_3111c6df35' ,'M_ID_48257bb851' ,'M_ID_4e461f7e14' ,'M_ID_50f575c681' ,'M_ID_57df19bf28' ,'M_ID_5ba019a379' ,'M_ID_5cea7e02a8' ,'M_ID_5e8220e564' ,'M_ID_620909e2a7' ,'M_ID_6f274b9340' ,'M_ID_77197a9b5e' ,'M_ID_77e2942cd8' ,'M_ID_79692349d6' ,'M_ID_82a30d9203' ,'M_ID_86be58d7e0' ,'M_ID_86ec983688' ,'M_ID_879f625c89' ,'M_ID_8c6b897b35' ,'M_ID_8fadd601d2' ,'M_ID_9139332ccc' ,'M_ID_940fb4498f' ,'M_ID_96ba3c7b60' ,'M_ID_97f0959810' ,'M_ID_9dbbaabb0a' ,'M_ID_9e84cda3b1' ,'M_ID_a9430074ad' ,'M_ID_b7459adc65' ,'M_ID_b794b9d9e8' ,'M_ID_b9dcf28cb9' ,'M_ID_c03b62d83d' ,'M_ID_c9855bf3e3' ,'M_ID_ca4d18b336' ,'M_ID_cc72dac895' ,'M_ID_cd2c0b07e9' ,'M_ID_d13262f4b0' ,'M_ID_de40fbd87c' ,'M_ID_deb43ff012' ,'M_ID_e020e9b302' ,'M_ID_e5374dabc0' ,'M_ID_ee75252e6a' ,'M_ID_eeba6b925d' ,'M_ID_ef87dd1879' ,'M_ID_f40ccc1312' ,'M_ID_f4be79819d' ,'M_ID_fc7d7969c3' ]

In [11]:
# tmp = dm.drop_duplicates().set_index(key)
# tmp = tmp[tmp.isin(m_id_list)]
ohe = pd.get_dummies(data=tmp).reset_index()
ohe = ohe.groupby(key).max().reset_index()
ohe.head()

,card_id,merchant_id_M_ID_00a6ca8a8a,merchant_id_M_ID_076cd8a829,merchant_id_M_ID_09d7a200d1,merchant_id_M_ID_1ac6bbc867,merchant_id_M_ID_1f4773aa76,merchant_id_M_ID_2637773dd2,merchant_id_M_ID_26d4fadb60,merchant_id_M_ID_29ec8d2ab8,merchant_id_M_ID_2c4b8c61f6,merchant_id_M_ID_3111c6df35,merchant_id_M_ID_48257bb851,merchant_id_M_ID_4e461f7e14,merchant_id_M_ID_50f575c681,merchant_id_M_ID_57df19bf28,merchant_id_M_ID_5ba019a379,merchant_id_M_ID_5cea7e02a8,merchant_id_M_ID_5e8220e564,merchant_id_M_ID_620909e2a7,merchant_id_M_ID_6f274b9340,merchant_id_M_ID_77197a9b5e,merchant_id_M_ID_77e2942cd8,merchant_id_M_ID_79692349d6,merchant_id_M_ID_82a30d9203,merchant_id_M_ID_86be58d7e0,merchant_id_M_ID_86ec983688,merchant_id_M_ID_879f625c89,merchant_id_M_ID_8c6b897b35,merchant_id_M_ID_8fadd601d2,merchant_id_M_ID_9139332ccc,merchant_id_M_ID_940fb4498f,merchant_id_M_ID_96ba3c7b60,merchant_id_M_ID_97f0959810,merchant_id_M_ID_9dbbaabb0a,merchant_id_M_ID_9e84cda3b1,merchant_id_M_ID_a9430074ad,merchant_id_M_ID_b7459adc65,merchant_id_M_ID_b794b9d9e8,merchant_id_M_ID_b9dcf28cb9,merchant_id_M_ID_c03b62d83d,merchant_id_M_ID_c9855bf3e3,merchant_id_M_ID_ca4d18b336,merchant_id_M_ID_cc72dac895,merchant_id_M_ID_cd2c0b07e9,merchant_id_M_ID_d13262f4b0,merchant_id_M_ID_de40fbd87c,merchant_id_M_ID_deb43ff012,merchant_id_M_ID_e020e9b302,merchant_id_M_ID_e5374dabc0,merchant_id_M_ID_ee75252e6a,merchant_id_M_ID_eeba6b925d,merchant_id_M_ID_ef87dd1879,merchant_id_M_ID_f40ccc1312,merchant_id_M_ID_f4be79819d,merchant_id_M_ID_fc7d7969c3
0,C_ID_00007093c1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,C_ID_0001238066,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
2,C_ID_0001506ef0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,C_ID_0001793786,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,C_ID_000183fdda,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
base = utils.read_df_pkl('../input/base*')
base_train = base[~base[target].isnull()].reset_index(drop=True)
base_test = base[base[target].isnull()].reset_index(drop=True)

df_train = base_train.merge(ohe, how='left', on=key)
df_test = base_test.merge(ohe, how='left', on=key)

df_feat = pd.concat([df_train, df_test], axis=0)

for col in df_feat.columns:
    if col in ignore_list:continue
    if not(col.count('M_ID')):continue
    feature = df_feat[col].values.astype('int8')
    utils.to_pkl_gzip(path = f'../features/1_first_valid/114_mer_{col}@', obj=feature)

100%|██████████| 3/3 [00:00<00:00, 82.26it/s]
